## CBOE SKEW analysis

In [50]:
import sys,os

if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
from volgrid import dgrid
from volgrid import create_voltables as cvt
import dash_core_components as dcc
import traceback

import plotly.graph_objs as go
from plotly.offline import iplot
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as pdr
import datetime
import pytz

#  do rest of imports
import dash
import dash_html_components as html
from dash.dependencies import Input, Output,State
import yfinance as yf
import pathlib
import pg_pandas as pg
from dateutil.relativedelta import *

### Define some interest rate and calendar functions

In [51]:
from pandas.tseries.holiday import USFederalHolidayCalendar
bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())
TIMEZONE = 'US/Eastern'

def get_rate_df(num_months_for_rate=1,start_datetime=None,end_datetime=None):
    # see if you can get Libor from the FRED API
    if start_datetime is None:
        n = datetime.datetime.now() - 252*bday_us #datetime.timedelta(7)
    else:
        if type(start_datetime)==int:
            sdt = start_datetime
            n = datetime.datetime(int(str(sdt)[0:4]),int(str(sdt)[4:6]), int(str(sdt)[6:8]))
        else:
            n = start_datetime 
    y = n.year
    m = n.month
    d = n.day
    beg = '%04d-%02d-%02d' %(y,m,d)
    ed = end_datetime if end_datetime is not None else (datetime.datetime.now() - datetime.timedelta(1))
    if type(ed)==int:
        ed = datetime.datetime(int(str(ed)[0:4]), int(str(ed)[4:6]), int(str(ed)[6:8]))
    y = ed.year
    m = ed.month
    d = ed.day
    eds = '%04d-%02d-%02d' %(y,m,d)
    fred_libor_file = f'USD{num_months_for_rate}MTD156N'
    df = pdr.DataReader(fred_libor_file, "fred", f'{beg}', f'{eds}')
    if len(df)<1:
        raise ValueError(f'FRED calendar of {fred_libor_file} does not contain dates {beg} through {eds}')
    df['fixed_rate'] = df[f'USD{num_months_for_rate}MTD156N'].astype(float)/100
    df.columns = df.columns.get_level_values(0)
    df['date_yyyymmdd'] = [int(d.year*100*100) + int(d.month*100) + int(d.day) for d in df.index]
    df.index = range(len(df))
    df['prate'] = df.shift(1).fixed_rate
    df['fixed_rate'] = df.apply(lambda r:r.prate if r.fixed_rate !=r.fixed_rate else r.fixed_rate,axis=1)
    return df[['date_yyyymmdd','fixed_rate']]


def get_rate(num_months_for_rate=1,rate_datetime=None):
    # see if you can get Libor from the FRED API
    if rate_datetime is None:
        n = datetime.datetime.now() - 7*bday_us #datetime.timedelta(7)
    else:
        n = rate_datetime #- datetime.timedelta(14)
    y = n.year
    m = n.month
    d = n.day
    beg = '%04d-%02d-%02d' %(y,m,d)
#     ed = n  + datetime.timedelta(1)
    ed = datetime.datetime.now()
    y = ed.year
    m = ed.month
    d = ed.day
    eds = '%04d-%02d-%02d' %(y,m,d)
    fred_libor_file = f'USD{num_months_for_rate}MTD156N'
    df = pdr.DataReader(fred_libor_file, "fred", f'{beg}', f'{eds}')
    if len(df)<1:
        raise ValueError(f'FRED calendar of {fred_libor_file} does not contain dates {beg} through {eds}')
#     fixed_rate = float(df.iloc[len(df)-1][f'USD{num_months_for_rate}MTD156N'])/100
    fixed_rate = float(df.iloc[-1][f'USD{num_months_for_rate}MTD156N'])/100
    return fixed_rate

def get_nth_weekday(year,month,target_weekday,nth_occurrence):
    '''
    weekday is the term that assigns numbers from 0 to 6 to the days of the weeks.
    weekday 0 = monday
    '''
    # get dayofweeks of year,month,1
    weekday_01 = datetime.datetime(year,month,1).weekday()
    if weekday_01 <= target_weekday:
        day_of_month_of_first_occurence = target_weekday - weekday_01
        day_of_month_of_nth_occurence = day_of_month_of_first_occurence + 1 + (nth_occurrence - 1) * 7
    else:
        day_of_month_of_nth_occurence = target_weekday - weekday_01 + 1 + (nth_occurrence) * 7 
    return datetime.datetime(year,month,day_of_month_of_nth_occurence)


MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_CODE = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}


def get_ES_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return get_nth_weekday(year,month,4,3)

def get_E6_expiry(symbol):
    monthcode_yy = symbol[2:]
    next_month = DICT_MONTH_CODE[monthcode_yy[0]] + 1
    year = 2000 + int(monthcode_yy[1:])
    if next_month>12:
        next_month = 1
        year += 1
    return datetime.datetime(year,next_month,1) - 7*bday_us

def get_CL_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month -1
    if month<1:
        month = 12
        year = year - 1
    return datetime.datetime(year,month,26) - 7*bday_us

def get_NG_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return datetime.datetime(year,month,1) - 4*bday_us

DICT_PRODUCT = {
    'E6':get_E6_expiry,
    'ES':get_ES_expiry,
    'CL':get_CL_expiry,
    'NG':get_NG_expiry,
}

    
def get_expiry(symbol):
    product = symbol[:2]
    f = DICT_PRODUCT[product]
    return f(symbol)


def dt_from_yyyymmdd(yyyymmdd,hour=0,minute=0,timezone=TIMEZONE):
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])  
    return datetime.datetime(y,m,d,hour,minute,tzinfo=pytz.timezone(timezone))

def yyyymmdd_from_dt(dt):
    y = int(dt.year)
    m = int(dt.month)
    d = int(dt.day)
    return y*100*100 + m*100 + d

def get_dte_pct(trade_yyyymmdd,expiry_yyyymmdd):
    dt_td = dt_from_yyyymmdd(trade_yyyymmdd)
    dt_xp = dt_from_yyyymmdd(expiry_yyyymmdd)
    return ((dt_xp - dt_td).days + 1)/365



### check dates vs last records in db for CL and ES

In [52]:
# pg.PgPandas??
pga = pg.PgPandas(dburl='127.0.0.1',username='',password='',databasename='sec_db')

/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



In [53]:
syms = ['NG'+m+yy for m in 'FGHJKMNQUVXZ' for yy in ['15','16','17','18','19']]
for s in syms:
    ge = str(get_expiry(s))
    exp1 = int(ge[0:4])*100*100 + int(ge[5:7])*100 + int(ge[8:10])
    
    sql = f'''
    select max(ot.settle_date) from sec_schema.options_table ot
    where ot.symbol='{s}' 
    ;
    '''
    df_options = pga.get_sql(sql)
    exp2 = df_options.iloc[0]['max']
    print(s,exp1,exp2,exp1==exp2)
    #assert(exp1==exp2)

NGF15 20141226 20141226 True
NGF16 20151228 20151228 True
NGF17 20161227 20161227 True
NGF18 20171226 20171226 True
NGF19 20181226 20181226 True
NGG15 20150127 20150127 True
NGG16 20160126 20160126 True
NGG17 20170126 20170126 True
NGG18 20180126 20180126 True
NGG19 20190128 20190128 True
NGH15 20150224 20150224 True
NGH16 20160224 20160224 True
NGH17 20170223 20170223 True
NGH18 20180223 20180223 True
NGH19 20190225 20190225 True
NGJ15 20150326 20150326 True
NGJ16 20160328 20160328 True
NGJ17 20170328 20170328 True
NGJ18 20180327 20180326 False
NGJ19 20190326 20190326 True
NGK15 20150427 20150427 True
NGK16 20160426 20160426 True
NGK17 20170425 20170425 True
NGK18 20180425 20180425 True
NGK19 20190425 20190425 True
NGM15 20150526 20150526 True
NGM16 20160525 20160525 True
NGM17 20170525 20170525 True
NGM18 20180525 20180525 True
NGM19 20190528 20190528 True
NGN15 20150625 20150625 True
NGN16 20160627 20160627 True
NGN17 20170627 20170627 True
NGN18 20180626 20180626 True
NGN19 2019062

In [54]:
get_rate(1)#,rate_datetime=datetime.datetime(2020,1,1))

2020-01-03 00:06:19,701 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): fred.stlouisfed.org:443
2020-01-03 00:06:25,781 - urllib3.connectionpool - DEBUG - https://fred.stlouisfed.org:443 "GET /graph/fredgraph.csv?id=USD1MTD156N HTTP/1.1" 200 34481


0.0180475

In [55]:
rdf = get_rate_df(1,datetime.datetime(2018,8,1),datetime.datetime(2018,8,31))
rdf

2020-01-03 00:06:30,283 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): fred.stlouisfed.org:443
2020-01-03 00:06:30,826 - urllib3.connectionpool - DEBUG - https://fred.stlouisfed.org:443 "GET /graph/fredgraph.csv?id=USD1MTD156N HTTP/1.1" 200 34481


,date_yyyymmdd,fixed_rate
0,20180801,0.020821
1,20180802,0.020802
2,20180803,0.020793
3,20180806,0.020826
4,20180807,0.020711
5,20180808,0.020634
6,20180809,0.020673
7,20180810,0.020667
8,20180813,0.020627
9,20180814,0.020635


### Paremeters to calculate SPX based SKEW

In [56]:
SKEW_TICKER = '^SPX' # yahoo symbol


In [57]:
df_skew = pd.read_csv('https://www.cboe.com/publish/scheduledtask/mktdata/datahouse/skewdailyprices.csv',header=1)

In [58]:
df_skew['date_split'] = df_skew.Date.apply(lambda s:s.split('/'))
df_skew['yyyymmdd'] = df_skew.date_split.apply(lambda s:int(s[2])*100*100 + int(s[0])*100 + int(s[1]))
df_skew

,Date,SKEW,Unnamed: 2,Unnamed: 3,date_split,yyyymmdd
0,1/2/1990,126.09,NaN,NaN,"[1, 2, 1990]",19900102
1,1/3/1990,123.34,NaN,NaN,"[1, 3, 1990]",19900103
2,1/4/1990,122.62,NaN,NaN,"[1, 4, 1990]",19900104
3,1/5/1990,121.27,NaN,NaN,"[1, 5, 1990]",19900105
4,1/8/1990,124.12,NaN,NaN,"[1, 8, 1990]",19900108
...,...,...,...,...,...,...
7544,12/26/2019,143.07,NaN,NaN,"[12, 26, 2019]",20191226
7545,12/27/2019,140.29,NaN,NaN,"[12, 27, 2019]",20191227
7546,12/30/2019,133.56,NaN,NaN,"[12, 30, 2019]",20191230
7547,12/31/2019,139.52,NaN,NaN,"[12, 31, 2019]",20191231


In [59]:
def get_closest_expiry_to_dt(dt=None,options_ticker=SKEW_TICKER):
    nn = dt
    if nn is None:
        nn = datetime.datetime.now()
    spx = yf.Ticker(options_ticker)
    dt_nth_weekday = get_nth_weekday(int(nn.year),int(nn.month),4,3)
    spx_expirys = [datetime.datetime(int(s[0:4]),int(s[5:7]),int(s[8:10])) for s in spx.options]
    diffs = [abs((se - dt_nth_weekday).days) for se in spx_expirys]
    index_of_closes_expiry = diffs.index(min(diffs))
    ret_exp = spx_expirys[index_of_closes_expiry]
    return ret_exp
def get_current_spx_3rd_friday_exp():
    nn = datetime.datetime.now()
    exp1 = get_closest_expiry_to_dt(dt=nn)
    dt_2nd_month = nn + relativedelta(months=+1)
    exp2 = get_closest_expiry_to_dt(dt=dt_2nd_month)
    dt_3rd_month = dt_2nd_month + relativedelta(months=+1)
    exp3 = get_closest_expiry_to_dt(dt=dt_3rd_month)    
    if (exp1 - nn).days < 10:
        return (exp2,exp3)
    return (exp1,exp2)
get_current_spx_3rd_friday_exp()

2020-01-03 00:06:45,439 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): query1.finance.yahoo.com:443
2020-01-03 00:06:45,927 - urllib3.connectionpool - DEBUG - https://query1.finance.yahoo.com:443 "GET /v7/finance/options/%5ESPX HTTP/1.1" 200 None
2020-01-03 00:06:46,997 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): query1.finance.yahoo.com:443
2020-01-03 00:06:47,312 - urllib3.connectionpool - DEBUG - https://query1.finance.yahoo.com:443 "GET /v7/finance/options/%5ESPX HTTP/1.1" 200 12264
2020-01-03 00:06:48,267 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): query1.finance.yahoo.com:443
2020-01-03 00:06:48,652 - urllib3.connectionpool - DEBUG - https://query1.finance.yahoo.com:443 "GET /v7/finance/options/%5ESPX HTTP/1.1" 200 12265


(datetime.datetime(2020, 1, 16, 0, 0), datetime.datetime(2020, 2, 20, 0, 0))

### Parmeters related to fetching current values of SPX

In [60]:
THRESHOLD_BID = .1 # minimum bid value when screening options
DELTA_K = 5 # strike difference between most of the options that get used to calculate SKEW
TRADE_DATE = 20191224
EXP_NEAR_YYYYMMDD = 20200116
EXP_FAR_YYYYMMDD = 20200220
INTEREST_RATE = (get_rate(1) + get_rate(2))/2 # calculate at run time ( this will be the average of the 1 month and 2 month libor rates from FRED)

OUTPUT_COLS = ['contractSymbol','strike','mid','dte_pct','ert',
                'forward_price','deltak','p1','p2','p3','e1','e2','e3']

DICT_THRESHOLD_BIDS = {'ES':1,'CL':.02,'CB':.02,'NG':.002}
DICT_DELTAK = {'ES':5,'CL':.5,'CB':.5,'NG':.1}


2020-01-03 00:06:52,053 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): fred.stlouisfed.org:443
2020-01-03 00:06:53,741 - urllib3.connectionpool - DEBUG - https://fred.stlouisfed.org:443 "GET /graph/fredgraph.csv?id=USD1MTD156N HTTP/1.1" 200 34481
2020-01-03 00:07:05,592 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): fred.stlouisfed.org:443
2020-01-03 00:07:07,551 - urllib3.connectionpool - DEBUG - https://fred.stlouisfed.org:443 "GET /graph/fredgraph.csv?id=USD2MTD156N HTTP/1.1" 200 33994


In [61]:
ed = str(EXP_FAR_YYYYMMDD)
dd = f'{ed[0:4]}-{ed[4:6]}-{ed[6:8]}'
c = yf.Ticker('^SPX')
df_t = c.option_chain(dd)
              

2020-01-03 00:07:13,799 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): query1.finance.yahoo.com:443
2020-01-03 00:07:15,121 - urllib3.connectionpool - DEBUG - https://query1.finance.yahoo.com:443 "GET /v7/finance/options/%5ESPX HTTP/1.1" 200 None
2020-01-03 00:07:15,535 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): query1.finance.yahoo.com:443
2020-01-03 00:07:16,564 - urllib3.connectionpool - DEBUG - https://query1.finance.yahoo.com:443 "GET /v7/finance/options/%5ESPX?date=1582243200 HTTP/1.1" 200 None


In [62]:
df_tp = df_t.puts[(df_t.puts.contractSymbol.str.slice(0,4)=='SPX2')]
df_tp[df_tp.strike>=3220.0].iloc[:3]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
228,SPX200221P03220000,2020-01-02 20:56:49,3220.0,40.10,46.1,46.6,-15.820000,-28.290415,3.0,460,0.133344,False,REGULAR,USD
230,SPX200221P03230000,2020-01-02 20:45:39,3230.0,45.16,49.2,49.6,-21.990002,-32.747580,1.0,143,0.130624,False,REGULAR,USD
231,SPX200221P03235000,2020-01-02 20:51:44,3235.0,45.55,50.8,51.2,-20.480000,-31.016203,2.0,100,0.129304,False,REGULAR,USD


In [63]:
df_tc = df_t.calls[(df_t.calls.bid>.1) & (df_t.calls.contractSymbol.str.slice(0,4)=='SPX2')]
df_tc[~df_tc.inTheMoney].iloc[:3]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
111,SPX200221C03260000,2020-01-02 21:13:03,3260.0,50.48,42.8,43.3,10.380001,25.885290,14.0,1087,0.092161,False,REGULAR,USD
113,SPX200221C03270000,2020-01-02 20:58:24,3270.0,43.17,37.2,37.6,8.329998,23.909292,7.0,231,0.090025,False,REGULAR,USD
116,SPX200221C03285000,2020-01-02 20:50:45,3285.0,33.25,29.5,29.9,7.450001,28.875975,4.0,426,0.087099,False,REGULAR,USD


In [64]:

def get_valid_series_from_yfinance(trade_date_yyyymmdd,expiry_yyyymmdd,ticker=SKEW_TICKER,threshold_bid=THRESHOLD_BID,interest_rate=None,deltak=DELTA_K):
#     # ****** Step 01: create interest rate to use if necessary
    assert(interest_rate is not None)
    ir = interest_rate
#     if ir is None:
#         ir  = (get_rate(1) + get_rate(2))/2
    # ******  Step 02:  get yfinance contract, and options series
    contract = yf.Ticker(ticker)
    # reformat expiry_yyyymmdd into something like 2019-12-20
    ed = str(expiry_yyyymmdd)
    date_string = f'{ed[0:4]}-{ed[4:6]}-{ed[6:8]}'
    # get the options chain from yahoo finance
    df_opt = contract.option_chain(date_string)

    # ****** Step 03: figure out forward price
    df_spx_p = df_opt.puts[(df_opt.puts.bid>threshold_bid) & (df_opt.puts.contractSymbol.str.slice(0,4)=='SPX2')].copy()
    df_spx_p['cp'] = 'p'
    
    # ******  Step 03:  extract out of the money puts
    df_spx_p = df_spx_p[~df_spx_p.inTheMoney]
    df_spx_p = df_spx_p.sort_values('strike')
    df_spx_p.index = range(len(df_spx_p))

    # ******  Step 04: extract out the ATM and the out of the money calls
    df_spx_c = df_opt.calls[(df_opt.calls.bid>threshold_bid) & (df_opt.calls.contractSymbol.str.slice(0,4)=='SPX2')].copy()
    return df_spx_c
    if len(df_spx_c) < 1:
        # if no bids, use strikes up to 10% out of the money
        last_put_strike = df_spx_p.iloc[-1].strike
        last_call_strike = round(last_put_strike*1.1,0)
        df_spx_c = df_opt.calls[(df_opt.calls.strike<=last_call_strike) & (df_opt.calls.contractSymbol.str.slice(0,4)=='SPX2')].copy()
        return df_spx_c
    df_spx_c['cp'] = 'c'
    df_spx_c = df_spx_c[~df_spx_c.inTheMoney]
    df_spx_c = df_spx_c.sort_values('strike')
    df_spx_c.index = range(len(df_spx_c))
    first_call_strike = df_spx_c.iloc[0].strike
    # save the lowest_itm_put for later calc of forward price
    call_strikes = df_spx_c.strike.values
    lowest_itm_put = df_spx_p[(df_spx_p.strike>=first_call_strike) & df_spx_p.strike.isin(call_strikes)].iloc[0]
    
    

    # ******  Step 05: merge puts and alls into one dataframe
    df_ret = df_spx_p.copy()
    df_ret = df_ret.append(df_spx_c)
    df_ret = df_ret.sort_values(['cp','strike'])
    df_ret.index = range(len(df_ret))
    df_ret['expiry_yyyymmdd'] = expiry_yyyymmdd
    df_ret['trade_date_yyyymmdd'] = trade_date_yyyymmdd
        
    # ******  Step 06: Add mid price, expiry, dte_pct, ert, forward price     
    df_ret['mid'] = (df_ret.bid + df_ret.ask)/2
    dte_pct = get_dte_pct(trade_date_yyyymmdd,expiry_yyyymmdd)
    df_ret['dte_pct'] = dte_pct
    ert = np.exp(dte_pct * ir)
    df_ret['ert'] = ert
    # find forward price by using the lowest strike call in df_ret 
    #     (what the SKEW whitepaper calls the At The Money option)
    # get ATM call midpoint
    atm_strike = lowest_itm_put.strike 
    
    atm_call = df_ret[(df_ret.strike==atm_strike) & (df_ret.cp=='c')].iloc[0]
    atm_call_price = (atm_call.bid + atm_call.ask) / 2 
    atm_put = lowest_itm_put
    atm_put_price = (atm_put.bid + atm_put.ask) / 2 
    forward_price = ert*(atm_call_price - atm_put_price) + atm_strike
    df_ret['forward_price'] = forward_price
    df_ret['k_over_fp'] = df_ret.strike / forward_price
    df_ret['deltak'] = deltak
    # create p1 unit values
    df_ret['p1']= df_ret.apply(lambda r: r.deltak / r.strike**2 * r.mid,axis=1)
    df_ret['p2'] = df_ret.apply(lambda r: 2 * r.p1 *(1-np.log(r.k_over_fp)),axis=1)
    df_ret['p3'] = df_ret.apply(lambda r: 3 * r.p1 * (2*np.log(r.k_over_fp) - np.log(r.k_over_fp)**2),axis=1)
    e1 = -(1+np.log(forward_price/atm_strike) - forward_price/atm_strike)
    e2 = 2 * np.log(atm_strike/forward_price) * (forward_price/atm_strike - 1) + 1/2 * np.log(atm_strike/forward_price)**2
    e3 = 3 * np.log(atm_strike/forward_price)**2 * (1/3 * np.log(atm_strike/forward_price) - 1 + forward_price/atm_strike)
    df_ret['e1'] = e1
    df_ret['e2'] = e2
    df_ret['e3'] = e3
    return df_ret
    

In [65]:
exp1,exp2 = get_current_spx_3rd_friday_exp()
td = datetime.datetime.now() - 7*bday_us
exp1_yyyymmdd = yyyymmdd_from_dt(exp1)
exp2_yyyymmdd = yyyymmdd_from_dt(exp2)

ir_now = (get_rate(1,td) + get_rate(2,td))/2 # calculate at run time ( this will be the average of the 1 month and 2 month libor rates from FRED)

df_series_near = get_valid_series_from_yfinance(td,exp1_yyyymmdd,interest_rate=ir_now)
df_series_far = get_valid_series_from_yfinance(td,exp2_yyyymmdd,interest_rate=ir_now)


2020-01-03 00:07:19,582 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): query1.finance.yahoo.com:443
2020-01-03 00:07:21,133 - urllib3.connectionpool - DEBUG - https://query1.finance.yahoo.com:443 "GET /v7/finance/options/%5ESPX HTTP/1.1" 200 12267
2020-01-03 00:07:22,805 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): query1.finance.yahoo.com:443
2020-01-03 00:07:29,938 - urllib3.connectionpool - DEBUG - https://query1.finance.yahoo.com:443 "GET /v7/finance/options/%5ESPX HTTP/1.1" 200 12264
2020-01-03 00:07:30,238 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): query1.finance.yahoo.com:443
2020-01-03 00:07:32,332 - urllib3.connectionpool - DEBUG - https://query1.finance.yahoo.com:443 "GET /v7/finance/options/%5ESPX HTTP/1.1" 200 12265
2020-01-03 00:07:34,788 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): fred.stlouisfed.org:443
2020-01-03 00:07:35,888 - urllib3.connectionpool - DEBUG - https://fr

In [66]:
# df_series_far[df_series_far.cp=='c'].iloc[0:1]

In [67]:
# df_series_near[df_series_near.cp=='p'].iloc[-1:]

### sum up p1's p2's and p3's and computer e1, e2, and e3', and create SKEW

In [68]:
def create_skew(df_series):
    e1 = df_series.iloc[0].e1
    e2 = df_series.iloc[0].e2
    e3 = df_series.iloc[0].e3
    ert = df_series.iloc[0].ert
    p1 = ert * -1 * df_series.p1.sum() + e1
    p2 = ert * df_series.p2.sum() + e2
    p3 = ert * df_series.p3.sum() + e3
    S_ =  (p3 - 3*p1*p2 + 2*p1**3) / (p2 - p1**2)**(3/2)
    SKEW = 100 - 10*S_
    return SKEW

In [69]:
# Tnear = df_series_near.iloc[0].dte_pct
# Tfar = df_series_far.iloc[0].dte_pct
# T30 = 30/365
# w = (Tfar-T30)/(Tfar-Tnear)
# skew_near = create_skew(df_series_near)
# skew_far = create_skew(df_series_far)
# final_skew = w*skew_near + (1-w) * skew_far
# print(final_skew,skew_near,skew_far,w)

### now do multiple days from Oct 2015

In [70]:
def get_valid_series_from_csv_file_df(df,threshold_bid=THRESHOLD_BID,interest_rate=INTEREST_RATE,deltak=DELTA_K):
    '''
    df: a DataFrame containing options settlements for a single day, and a single expiry
    '''
    # ****** Step 01: create interest rate to use if necessary
    ir = interest_rate
#     if ir is None:
#         ir  = (get_rate(1) + get_rate(2))/2
    
    df_ret = df.copy()
    trade_date_yyyymmdd = df_ret.trade_date_yyyymmdd.iloc[0]
    expiry_yyyymmdd = df_ret.expiry_yyyymmdd.iloc[0]
    assert(len(df_ret.expiry_yyyymmdd.unique())==1)
    # ******  Step 02: Add mid price, expiry, dte_pct, ert, forward price     
    df_ret['mid'] = (df_ret.bid + df_ret.ask)/2
    dte_pct = get_dte_pct(trade_date_yyyymmdd,expiry_yyyymmdd)
    df_ret['dte_pct'] = dte_pct
    ert = np.exp(dte_pct * ir)
    df_ret['ert'] = ert
    lowest_atm_call = df_ret[(df_ret.underlying_last<df_ret.strike) & (df_ret.cp=='c')].sort_values('strike').iloc[0]
    atm_call_price = lowest_atm_call.mid
    atm_strike = lowest_atm_call.strike
    atm_put_price = df_ret[(df_ret.strike==atm_strike) & (df_ret.cp=='p')].iloc[0].mid
    forward_price = ert*(atm_call_price - atm_put_price) + atm_strike
    
    # ******  Step 03: Limit options to out of the money puts, the ATM call and out of the money calls
    df_ret_c = df_ret[(df_ret.cp=='c') & (df_ret.strike>=atm_strike)]
    df_ret_p = df_ret[(df_ret.cp=='p') & (df_ret.strike<atm_strike)]
    df_ret = df_ret_c.append(df_ret_p).sort_values(['cp','strike'])
    # ******  Step 04: make sure options have minimum bid
    df_ret = df_ret[df_ret.bid>threshold_bid]
    
    
    df_ret['forward_price'] = forward_price
    df_ret['k_over_fp'] = df_ret.strike / forward_price
    df_ret['deltak'] = deltak
    # create p1 unit values
    df_ret['p1']= df_ret.apply(lambda r: r.deltak / r.strike**2 * r.mid,axis=1)
    df_ret['p2'] = df_ret.apply(lambda r: 2 * r.p1 *(1-np.log(r.k_over_fp)),axis=1)
    df_ret['p3'] = df_ret.apply(lambda r: 3 * r.p1 * (2*np.log(r.k_over_fp) - np.log(r.k_over_fp)**2),axis=1)
    e1 = -(1+np.log(forward_price/atm_strike) - forward_price/atm_strike)
    e2 = 2 * np.log(atm_strike/forward_price) * (forward_price/atm_strike - 1) + 1/2 * np.log(atm_strike/forward_price)**2
    e3 = 3 * np.log(atm_strike/forward_price)**2 * (1/3 * np.log(atm_strike/forward_price) - 1 + forward_price/atm_strike)
    df_ret['e1'] = e1
    df_ret['e2'] = e2
    df_ret['e3'] = e3

    return df_ret
    

In [71]:
def add_days_to_yyyymmdd(yyyymmdd,days_to_add):
    s = str(yyyymmdd)
    y = int(s[0:4])
    m = int(s[4:6])
    d = int(s[6:8])
    dt = datetime.datetime(y,m,d) + datetime.timedelta(days_to_add)
    ret = int(dt.year)*100*100 + int(dt.month)*100 + int(dt.day)
    return ret

In [72]:
home =  pathlib.Path.home()
df_spx_oct_2015= pd.read_csv(f'{home}/downloads/spx_20151001_to_20151030.csv')
df_spx_oct_2015 = df_spx_oct_2015[df_spx_oct_2015.underlying=='SPX']
df_spx_oct_2015['cp'] = df_spx_oct_2015['type'].apply(lambda v:v[0])
df_spx_oct_2015['trade_date_yyyymmdd'] = df_spx_oct_2015.quotedate.apply(lambda s:s.split('/'))
df_spx_oct_2015['trade_date_yyyymmdd'] = df_spx_oct_2015.trade_date_yyyymmdd.apply(lambda s:(2000 + int(s[2]))*100*100 + int(s[0])*100 + int(s[1]))
df_spx_oct_2015['expiry_yyyymmdd'] = df_spx_oct_2015.expiration.apply(lambda s:s.split('/'))
df_spx_oct_2015['expiry_yyyymmdd'] = df_spx_oct_2015.expiry_yyyymmdd.apply(lambda s:(2000 + int(s[2]))*100*100 + int(s[0])*100 + int(s[1]))

df_spx_oct_2015.tail()


,underlying,underlying_last,exchange,optionroot,optionext,type,expiration,quotedate,strike,last,...,openinterest,impliedvol,delta,gamma,theta,vega,optionalias,cp,trade_date_yyyymmdd,expiry_yyyymmdd
233689,SPX,2084.58,*,SPX171215P02750000,NaN,put,12/15/17,10/30/15,2750,0.00,...,0,0.1993,-0.7836,0.0005,-34.2409,891.7503,SPX171215P02750000,p,20151030,20171215
233690,SPX,2084.58,*,SPX171215P02800000,NaN,put,12/15/17,10/30/15,2800,0.00,...,0,0.2036,-0.7945,0.0005,-33.6388,865.0028,SPX171215P02800000,p,20151030,20171215
233691,SPX,2084.58,*,SPX171215P02900000,NaN,put,12/15/17,10/30/15,2900,0.00,...,0,0.2134,-0.8114,0.0004,-33.0113,821.1380,SPX171215P02900000,p,20151030,20171215
233692,SPX,2084.58,*,SPX171215P03000000,NaN,put,12/15/17,10/30/15,3000,917.55,...,18,0.2238,-0.8241,0.0004,-32.7816,786.1406,SPX171215P03000000,p,20151030,20171215
233693,SPX,2084.58,*,SPX171215P03500000,NaN,put,12/15/17,10/30/15,3500,1585.70,...,69,0.2685,-0.8665,0.0003,-30.9119,654.9595,SPX171215P03500000,p,20151030,20171215


In [73]:
df_spx_oct_2015[df_spx_oct_2015.trade_date_yyyymmdd==20151001]

,underlying,underlying_last,exchange,optionroot,optionext,type,expiration,quotedate,strike,last,...,openinterest,impliedvol,delta,gamma,theta,vega,optionalias,cp,trade_date_yyyymmdd,expiry_yyyymmdd
0,SPX,1921.42,*,SPX151016C00400000,NaN,call,10/16/15,10/1/15,400,0.00,...,0,0.2589,1.0000,0.0000,-1.2854,0.0000,SPX151016C00400000,c,20151001,20151016
1,SPX,1921.42,*,SPX151016C00500000,NaN,call,10/16/15,10/1/15,500,0.00,...,0,0.2589,1.0000,0.0000,-1.6068,0.0000,SPX151016C00500000,c,20151001,20151016
2,SPX,1921.42,*,SPX151016C00600000,NaN,call,10/16/15,10/1/15,600,0.00,...,0,0.2589,1.0000,0.0000,-1.9282,0.0000,SPX151016C00600000,c,20151001,20151016
3,SPX,1921.42,*,SPX151016C00700000,NaN,call,10/16/15,10/1/15,700,0.00,...,0,0.2589,1.0000,0.0000,-2.2495,0.0000,SPX151016C00700000,c,20151001,20151016
4,SPX,1921.42,*,SPX151016C00750000,NaN,call,10/16/15,10/1/15,750,0.00,...,0,0.2589,1.0000,0.0000,-2.4102,0.0000,SPX151016C00750000,c,20151001,20151016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2879,SPX,1921.42,*,SPX171215P02750000,NaN,put,12/15/17,10/1/15,2750,0.00,...,0,0.2267,-0.8093,0.0004,-32.0924,776.2559,SPX171215P02750000,p,20151001,20171215
2880,SPX,1921.42,*,SPX171215P02800000,NaN,put,12/15/17,10/1/15,2800,0.00,...,0,0.2311,-0.8172,0.0004,-31.6166,756.2410,SPX171215P02800000,p,20151001,20171215
2881,SPX,1921.42,*,SPX171215P02900000,NaN,put,12/15/17,10/1/15,2900,0.00,...,0,0.2408,-0.8295,0.0004,-31.1105,723.5832,SPX171215P02900000,p,20151001,20171215
2882,SPX,1921.42,*,SPX171215P03000000,NaN,put,12/15/17,10/1/15,3000,917.55,...,18,0.2509,-0.8389,0.0003,-30.9070,697.4111,SPX171215P03000000,p,20151001,20171215


### Get unique quotedate's

In [74]:
trade_dates_yyyymmdd = df_spx_oct_2015.trade_date_yyyymmdd.unique()
dict_df_exp_pair = {}
for d in trade_dates_yyyymmdd:
    first_expiry_cutoff = add_days_to_yyyymmdd(d,10)
    df_spx_1 = df_spx_oct_2015[(df_spx_oct_2015.trade_date_yyyymmdd==d) & (df_spx_oct_2015.expiry_yyyymmdd>first_expiry_cutoff)]
    first_2_expiries = df_spx_1.expiry_yyyymmdd.unique()[:2]
    df_spx_exp1 = df_spx_1[df_spx_1.expiry_yyyymmdd==first_2_expiries[0]]
    df_spx_exp1_with_all_fields = get_valid_series_from_csv_file_df(df_spx_exp1)
    df_spx_exp2 = df_spx_1[df_spx_1.expiry_yyyymmdd==first_2_expiries[1]]
    df_spx_exp2_with_all_fields = get_valid_series_from_csv_file_df(df_spx_exp2)

    Tnear = df_spx_exp1_with_all_fields.iloc[0].dte_pct
    Tfar = df_spx_exp2_with_all_fields.iloc[0].dte_pct
    T30 = 30/365
    w = (Tfar-T30)/(Tfar-Tnear)
    skew_near = create_skew(df_spx_exp1_with_all_fields)
    skew_far = create_skew(df_spx_exp2_with_all_fields)
    final_skew = w*skew_near + (1-w) * skew_far
    actual_skew = df_skew[df_skew.yyyymmdd==d].iloc[0].SKEW
    print(d,actual_skew,final_skew,skew_near,skew_far,w,first_2_expiries[0],first_2_expiries[1])    

    dict_df_exp_pair[d] = {
        'df_spx_1':df_spx_exp1_with_all_fields,
                           'df_spx_2':df_spx_exp2_with_all_fields,
                           'skew_near':skew_near,
                           'skew_far':skew_far,
                           'final_skew':final_skew
    }
    


20151001 124.81 117.39830537846379 114.34102867805181 121.98422042908179 0.6000000000000001 20151016 20151120
20151002 115.75 118.34807534855344 116.04113356267328 121.42399772972695 0.5714285714285714 20151016 20151120
20151005 132.83 120.13858131687212 116.5783242328683 123.5010463406535 0.48571428571428577 20151016 20151120
20151006 129.32 126.83758856884972 124.7898436310071 121.07830593116734 1.5517241379310347 20151120 20151219
20151007 132.39 126.87766640519298 124.86550608922518 120.97532947835407 1.5172413793103448 20151120 20151219
20151008 136.69 130.50095697899576 128.552180199171 124.51542829810548 1.4827586206896552 20151120 20151219
20151009 134.66 130.42865653922496 129.20871236670632 126.4872984433955 1.4482758620689657 20151120 20151219
20151012 137.25 129.36173824371002 128.4626884006672 125.85544385584299 1.3448275862068966 20151120 20151219
20151013 133.81 129.26015974804105 128.5530677007974 126.27465999301239 1.310344827586207 20151120 20151219
20151014 135.82 12

### Now use ES contract history from sec_db database

In [75]:
s = 'CLZ18'
d = 20180801
sql = f'''
select * from sec_schema.options_table ot
where ot.settle_date>={d} 
and ot.symbol='{s}'
;
'''
df_options = pga.get_sql(sql)


sql = f'''
select * from sec_schema.underlying_table ft 
where ft.settle_date='{d}' 
and ft.symbol='{s}'
;
'''
df_futures = pga.get_sql(sql)
df_options['forward_price'] = df_futures.iloc[0].close
df_options['trade_date_yyyymmdd'] = df_options.settle_date

print(df_options)

      symbol  strike pc  settle_date  open   high    low  close  adj_close  \
0      CLZ18   100.0  C     20180801   0.0   0.03   0.03   0.03       0.03   
1      CLZ18   100.0  P     20180801   0.0  34.22  34.22  34.22      34.22   
2      CLZ18    15.0  P     20180801   0.0   0.01   0.01   0.01       0.01   
3      CLZ18    20.0  C     20180801   0.0  47.80  45.78  45.78      45.78   
4      CLZ18    20.0  P     20180801   0.0   0.01   0.01   0.01       0.01   
...      ...     ... ..          ...   ...    ...    ...    ...        ...   
20302  CLZ18    97.5  C     20181114   0.0   0.01   0.01   0.01       0.01   
20303  CLZ18    98.0  C     20181114   0.0   0.01   0.01   0.01       0.01   
20304  CLZ18    98.5  C     20181114   0.0   0.01   0.01   0.01       0.01   
20305  CLZ18    99.0  C     20181114   0.0   0.01   0.01   0.01       0.01   
20306  CLZ18    99.5  C     20181114   0.0   0.01   0.01   0.01       0.01   

       volume  open_interest  forward_price  trade_date_yyyymmd

In [76]:
df_options

,symbol,strike,pc,settle_date,open,high,low,close,adj_close,volume,open_interest,forward_price,trade_date_yyyymmdd
0,CLZ18,100.0,C,20180801,0.0,0.03,0.03,0.03,0.03,36,13307,65.78,20180801
1,CLZ18,100.0,P,20180801,0.0,34.22,34.22,34.22,34.22,0,974,65.78,20180801
2,CLZ18,15.0,P,20180801,0.0,0.01,0.01,0.01,0.01,0,3,65.78,20180801
3,CLZ18,20.0,C,20180801,0.0,47.80,45.78,45.78,45.78,1,5,65.78,20180801
4,CLZ18,20.0,P,20180801,0.0,0.01,0.01,0.01,0.01,0,641,65.78,20180801
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20302,CLZ18,97.5,C,20181114,0.0,0.01,0.01,0.01,0.01,0,1259,65.78,20181114
20303,CLZ18,98.0,C,20181114,0.0,0.01,0.01,0.01,0.01,0,2013,65.78,20181114
20304,CLZ18,98.5,C,20181114,0.0,0.01,0.01,0.01,0.01,0,1155,65.78,20181114
20305,CLZ18,99.0,C,20181114,0.0,0.01,0.01,0.01,0.01,0,3145,65.78,20181114


### Create date math functions to get options/futures expiry dates

In [77]:
tt = f'./opvoct15/opv10015.csv'
import io
def _df_from_text(text_path):
    lines = open(text_path,'r').read()
    options_header = 'contract,month_year,strike_right,date,open,high,low,close,volume,open_interest'
    lines2 = options_header + '\n'+ lines
    lines3 = lines2.split('\n')
    s2 = io.StringIO()
    for l in lines3:
        s2.write(l+'\n')
    s2.seek(0)
    df = pd.read_csv(s2)
    return df
df_tt = _df_from_text(tt)

In [78]:
[s for s in sorted(df_tt.contract.unique()) if s[0]=='E']

['E6', 'ES', 'EW']

In [79]:

df_tt[df_tt.contract.str.contains('ES')].month_year.unique()

array(['V2015', 'X2015', 'Z2015', 'F2016', 'H2016', 'M2016', 'U2016'],
      dtype=object)

In [80]:
#" ".join(sorted(list(set([s[:4] for s in df_tt[df_tt.contract.str.contains('ES')].strike_right.unique()]))))

In [81]:
# import datetime as dt

# from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
#     USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
#     USLaborDay, USThanksgivingDay


# class USTradingCalendar(AbstractHolidayCalendar):
#     rules = [
#         Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
#         USMartinLutherKingJr,
#         USPresidentsDay,
#         GoodFriday,
#         USMemorialDay,
#         Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
#         USLaborDay,
#         USThanksgivingDay,
#         Holiday('Christmas', month=12, day=25, observance=nearest_workday)
#     ]


# def get_trading_close_holidays(year):
#     inst = USTradingCalendar()

#     return inst.holidays(dt.datetime(year-1, 12, 31), dt.datetime(year, 12, 31))


# print(get_trading_close_holidays(2014)) 

### get ESX15 and ESZ15 options, ESZ15 underlying to 

In [82]:
s = 'ESV15'
d = 20151001
sql = f'''
select * from sec_schema.options_table ot
where ot.symbol='{s}' and settle_date={d}
;
'''
df_options = pga.get_sql(sql)

f = 'ESZ15'
sql = f'''
select * from sec_schema.underlying_table ot
where ot.symbol='{f}'  and settle_date={d}
;
'''
df_futures = pga.get_sql(sql)
atm = df_futures.iloc[0].close



In [83]:
df_options

,symbol,strike,pc,settle_date,open,high,low,close,adj_close,volume,open_interest
0,ESV15,500.0,C,20151001,0.0,1416.80,1416.75,1416.75,1416.75,0,0
1,ESV15,500.0,P,20151001,0.0,0.05,0.01,0.01,0.01,0,0
2,ESV15,600.0,C,20151001,0.0,1316.80,1316.75,1316.75,1316.75,0,0
3,ESV15,600.0,P,20151001,0.0,0.05,0.01,0.01,0.01,0,0
4,ESV15,800.0,C,20151001,0.0,1116.80,1116.75,1116.75,1116.75,0,0
...,...,...,...,...,...,...,...,...,...,...,...
507,ESV15,3100.0,P,20151001,0.0,1183.25,1183.20,1183.25,1183.25,0,0
508,ESV15,3125.0,C,20151001,0.0,0.05,0.01,0.01,0.01,0,0
509,ESV15,3125.0,P,20151001,0.0,1208.25,1208.20,1208.25,1208.25,0,0
510,ESV15,3150.0,C,20151001,0.0,0.05,0.01,0.01,0.01,0,0


In [84]:
df_futures

,symbol,settle_date,contract_num,open,high,low,close,adj_close,volume,open_interest
0,ESZ15,20151001,1,1902.5,1929.5,1890.25,1916.75,1916.75,2216643,2867181


In [85]:
dfc = df_options[(df_options.pc.str.lower()=='c')][['settle_date','strike','close','pc']].sort_values('strike')
dfc = dfc[dfc.strike >= atm]
dfp = df_options[(df_options.pc.str.lower()=='p')][['settle_date','strike','close','pc']].sort_values('strike')
dfp = dfp[dfp.strike < atm]
dfb = dfc.append(dfp).sort_values(['pc','strike'])
dfb.pc = dfb.pc.str.lower()
dfb = dfb.rename(columns={'pc':'cp','settle_date':'trade_date_yyyymmdd'})
dfb = dfb[dfb.close >= 1 ]
dfb['expiry_yyyymmdd']  = get_expiry('ESV15')
dfb.strike.min(),dfb.strike.max(),len(dfb)

(1680.0, 2025.0, 70)

In [86]:
dfp.tail()

,settle_date,strike,close,pc
239,20151001,1895.0,24.75,P
241,20151001,1900.0,26.50,P
243,20151001,1905.0,28.25,P
245,20151001,1910.0,30.00,P
247,20151001,1915.0,32.00,P


### create SKEW from ESV15

In [87]:
ir_20151001 = get_rate(1,dt_from_yyyymmdd(20151001))

2020-01-03 00:07:58,098 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): fred.stlouisfed.org:443
2020-01-03 00:07:58,560 - urllib3.connectionpool - DEBUG - https://fred.stlouisfed.org:443 "GET /graph/fredgraph.csv?id=USD1MTD156N HTTP/1.1" 200 34481


In [88]:
dict_fut_mon = {'F':'H','G':'H','H':'H','J':'M','K':'M','M':'M','N':'U','Q':'U','U':'U','V':'Z','X':'Z','Z':'Z'}
def get_valid_series_from_barchartacs(symbol,trade_date_yyyymmdd,
                                      interest_rate=None,deltak=DELTA_K):
    threshold_bid = DICT_THRESHOLD_BIDS[symbol[:2]]
#     print(symbol,trade_date_yyyymmdd)
    ir = interest_rate 
    if ir is None:
        ir = get_rate(1,dt_from_yyyymmdd(trade_date_yyyymmdd))
        
#     # ****** Step 01: get futures data from sql
    fm = dict_fut_mon[symbol[2]]
    futures_symbol = symbol[0:2] + fm + symbol[3:]
    sql = f'''
    select * from sec_schema.underlying_table ot
    where ot.symbol='{futures_symbol}'  and settle_date={trade_date_yyyymmdd}
    ;
    '''
    df_futures = pga.get_sql(sql)
    forward_price = df_futures.iloc[0].close

#     # ****** Step 02: get options data from sql
    sql = f'''
    select * from sec_schema.options_table ot
    where ot.symbol='{symbol}'  and settle_date={trade_date_yyyymmdd}
    ;
    '''
    df_options = pga.get_sql(sql)
    dfc = df_options[(df_options.pc.str.lower()=='c')][['settle_date','strike','close','pc']].sort_values('strike')
    dfc = dfc[dfc.strike >= forward_price]
    dfp = df_options[(df_options.pc.str.lower()=='p')][['settle_date','strike','close','pc']].sort_values('strike')
    dfp = dfp[dfp.strike < forward_price]
    atm_strike = dfc[dfc.strike == dfc.strike.min()].iloc[0].strike
    
    # ******  Step 03: merge puts and alls into one dataframe
    dfb = dfc.append(dfp).sort_values(['pc','strike'])    
    dfb.pc = dfb.pc.str.lower()
    dfb = dfb.rename(columns={'pc':'cp','settle_date':'trade_date_yyyymmdd','close':'mid'})
    dfb = dfb[dfb.mid >= threshold_bid ]
    exp_dt  = get_expiry(symbol)
    expiry_yyyymmdd = int(exp_dt.year)*100*100 + int(exp_dt.month)*100 + int(exp_dt.day)
    dfb['expiry_yyyymmdd'] = expiry_yyyymmdd
    df_ret = dfb.copy()
    df_ret['trade_date_yyyymmdd'] = trade_date_yyyymmdd
    df_ret['forward_price'] = forward_price
    
    df_ret = df_ret.sort_values(['cp','strike'])
    df_ret.index = range(len(df_ret))

    # ******  Step 04: Add m dte_pct, ert  
    dte_pct = get_dte_pct(trade_date_yyyymmdd,expiry_yyyymmdd)
    df_ret['dte_pct'] = dte_pct
    ert = np.exp(dte_pct * ir)
    df_ret['ert'] = ert
    
    df_ret['k_over_fp'] = df_ret.strike / forward_price
    df_ret['deltak'] = deltak
    
    # ****** Step 05: create p1,p2,p3, e1,e2,e3 unit values
    df_ret['p1']= df_ret.apply(lambda r: r.deltak / r.strike**2 * r.mid,axis=1)
    df_ret['p2'] = df_ret.apply(lambda r: 2 * r.p1 *(1-np.log(r.k_over_fp)),axis=1)
    df_ret['p3'] = df_ret.apply(lambda r: 3 * r.p1 * (2*np.log(r.k_over_fp) - np.log(r.k_over_fp)**2),axis=1)
    e1 = -(1+np.log(forward_price/atm_strike) - forward_price/atm_strike)
    e2 = 2 * np.log(atm_strike/forward_price) * (forward_price/atm_strike - 1) + 1/2 * np.log(atm_strike/forward_price)**2
    e3 = 3 * np.log(atm_strike/forward_price)**2 * (1/3 * np.log(atm_strike/forward_price) - 1 + forward_price/atm_strike)
    df_ret['e1'] = e1
    df_ret['e2'] = e2
    df_ret['e3'] = e3
    return df_ret
    

In [89]:
get_valid_series_from_barchartacs('NGM19',20190401,interest_rate=.02,deltak=.1)


,trade_date_yyyymmdd,strike,mid,cp,expiry_yyyymmdd,forward_price,dte_pct,ert,k_over_fp,deltak,p1,p2,p3,e1,e2,e3
0,20190401,2.75,0.087,c,20190528,2.749,0.158904,1.003183,1.000364,0.1,0.001150,0.002300,0.000003,6.613173e-08,-1.983712e-07,-9.619453e-11
1,20190401,2.80,0.064,c,20190528,2.749,0.158904,1.003183,1.018552,0.1,0.000816,0.001603,0.000089,6.613173e-08,-1.983712e-07,-9.619453e-11
2,20190401,2.85,0.046,c,20190528,2.749,0.158904,1.003183,1.036741,0.1,0.000566,0.001092,0.000120,6.613173e-08,-1.983712e-07,-9.619453e-11
3,20190401,2.90,0.033,c,20190528,2.749,0.158904,1.003183,1.054929,0.1,0.000392,0.000743,0.000123,6.613173e-08,-1.983712e-07,-9.619453e-11
4,20190401,2.95,0.023,c,20190528,2.749,0.158904,1.003183,1.073117,0.1,0.000264,0.000491,0.000108,6.613173e-08,-1.983712e-07,-9.619453e-11
5,20190401,3.00,0.016,c,20190528,2.749,0.158904,1.003183,1.091306,0.1,0.000178,0.000324,0.000089,6.613173e-08,-1.983712e-07,-9.619453e-11
6,20190401,3.05,0.011,c,20190528,2.749,0.158904,1.003183,1.109494,0.1,0.000118,0.000212,0.000070,6.613173e-08,-1.983712e-07,-9.619453e-11
7,20190401,3.10,0.008,c,20190528,2.749,0.158904,1.003183,1.127683,0.1,0.000083,0.000146,0.000056,6.613173e-08,-1.983712e-07,-9.619453e-11
8,20190401,3.15,0.006,c,20190528,2.749,0.158904,1.003183,1.145871,0.1,0.000060,0.000104,0.000046,6.613173e-08,-1.983712e-07,-9.619453e-11
9,20190401,3.20,0.004,c,20190528,2.749,0.158904,1.003183,1.164060,0.1,0.000039,0.000066,0.000033,6.613173e-08,-1.983712e-07,-9.619453e-11


### Loop through a set of days and calculate skew

In [90]:
trade_date_yyyymmdd_beg = 20180701
trade_date_yyyymmdd_end = 20180810
df_rates = get_rate_df(1,trade_date_yyyymmdd_beg,trade_date_yyyymmdd_end)
symbol='ESU18'
futures_symbol = 'ES' + dict_fut_mon[symbol[2]] + symbol[3:]
sql = f'''
select * from sec_schema.underlying_table ot
where ot.symbol='{futures_symbol}'  and settle_date>={trade_date_yyyymmdd_beg} and settle_date <= {trade_date_yyyymmdd_end}
;
'''
trade_dates_yyyymmdd = pga.get_sql(sql).settle_date.values
dates = []
actual_skews = []
final_skews  = []
dict_df_exp_pair = {}
for d in trade_dates_yyyymmdd:
    ir = df_rates[df_rates.date_yyyymmdd==d].iloc[0].fixed_rate
    dfes = get_valid_series_from_barchartacs(symbol,d,interest_rate=ir,deltak=5)
    final_skew = create_skew(dfes)
    actual_skew = df_skew[df_skew.yyyymmdd==d].iloc[0].SKEW
    print(d,actual_skew,final_skew) 
    dates.append(d)
    actual_skews.append(actual_skew)
    final_skews.append(final_skew)
    dict_df_exp_pair[d] = {
        'df_spx_1':dfes.copy(),
                           'final_skew':final_skew
    }
df_pga_skew = pd.DataFrame({'date':dates,'actual_skew':actual_skews,'final_skew':final_skews})    


2020-01-03 00:07:59,426 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): fred.stlouisfed.org:443
2020-01-03 00:07:59,935 - urllib3.connectionpool - DEBUG - https://fred.stlouisfed.org:443 "GET /graph/fredgraph.csv?id=USD1MTD156N HTTP/1.1" 200 34481


20180702 140.89 127.28261462660657
20180703 134.57 126.47402498067629
20180705 139.86 128.47362125785867
20180706 141.02 128.91267208302418
20180709 138.96 129.5309546164385
20180710 139.26 129.69705348834498
20180711 136.85 129.3790872246444
20180712 138.22 130.06752109784992
20180713 142.48 130.4361453965027
20180716 148.16 131.1090145516708
20180717 149.58 132.64725208928502
20180718 154.25 133.6824841601792
20180719 150.98 133.27409019158233
20180720 152.72 133.9253638709254
20180723 153.78 133.8492600325658
20180724 145.94 133.95821921884732
20180725 145.09 135.09510148149667
20180726 142.18 134.90824521249664
20180727 144.57 136.5999953701361
20180730 141.57 136.18156576701372
20180731 144.49 136.5738516738254
20180801 142.77 136.00111746066636
20180802 143.83 135.3567366513231
20180803 146.19 136.21393714306723
20180806 142.01 133.0224930854425
20180807 138.94 131.67080199735346
20180808 139.84 130.64705543131583
20180809 138.61 131.48611812836026
20180810 149.2 136.895966922653

In [91]:
df_pga_skew

,date,actual_skew,final_skew
0,20180702,140.89,127.282615
1,20180703,134.57,126.474025
2,20180705,139.86,128.473621
3,20180706,141.02,128.912672
4,20180709,138.96,129.530955
5,20180710,139.26,129.697053
6,20180711,136.85,129.379087
7,20180712,138.22,130.067521
8,20180713,142.48,130.436145
9,20180716,148.16,131.109015


In [92]:
iplot(cvt.plotly_plot(df_pga_skew,'date',plot_title='skew'))

In [93]:
def create_skew_df(symbol,beg_yyyymmdd, end_yyyymmdd,deltak=5):
    trade_date_yyyymmdd_beg = beg_yyyymmdd
    trade_date_yyyymmdd_end = end_yyyymmdd
    df_rates = get_rate_df(1,trade_date_yyyymmdd_beg,trade_date_yyyymmdd_end)
    futures_symbol = symbol[:(len(symbol)-3)] + dict_fut_mon[symbol[2]] + symbol[3:]
    sql = f'''
    select * from sec_schema.underlying_table ot
    where ot.symbol='{futures_symbol}'  and settle_date>={trade_date_yyyymmdd_beg} and settle_date <= {trade_date_yyyymmdd_end}
    ;
    '''
    trade_dates_yyyymmdd = pga.get_sql(sql).settle_date.values
    dates = []
    final_skews  = []
    dict_df_exp_pair = {}
    for d in trade_dates_yyyymmdd:
        ir = df_rates[df_rates.date_yyyymmdd==d].iloc[0].fixed_rate
        dfes = get_valid_series_from_barchartacs(symbol,d,interest_rate=ir,deltak=deltak)
        final_skew = create_skew(dfes)
        dates.append(d)
        final_skews.append(final_skew)
        dict_df_exp_pair[d] = {
            'df_spx_1':dfes.copy(),
                               'final_skew':final_skew
        }
    df_pga_skew = pd.DataFrame({'date':dates,'final_skew':final_skews})    
    return df_pga_skew

In [94]:
get_valid_series_from_barchartacs('CLF20',20191001,interest_rate=get_rate(1,datetime.datetime(2019,10,1)),deltak=.1)

2020-01-03 00:08:05,552 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): fred.stlouisfed.org:443
2020-01-03 00:08:07,474 - urllib3.connectionpool - DEBUG - https://fred.stlouisfed.org:443 "GET /graph/fredgraph.csv?id=USD1MTD156N HTTP/1.1" 200 34481


,trade_date_yyyymmdd,strike,mid,cp,expiry_yyyymmdd,forward_price,dte_pct,ert,k_over_fp,deltak,p1,p2,p3,e1,e2,e3
0,20191001,53.0,3.57,c,20191216,52.54,0.210959,1.003815,1.008755,0.1,0.000127,0.000252,6.618265e-06,0.000038,-0.000113,-0.000001
1,20191001,53.5,3.31,c,20191216,52.54,0.210959,1.003815,1.018272,0.1,0.000116,0.000227,1.244988e-05,0.000038,-0.000113,-0.000001
2,20191001,54.0,3.07,c,20191216,52.54,0.210959,1.003815,1.027788,0.1,0.000105,0.000205,1.707680e-05,0.000038,-0.000113,-0.000001
3,20191001,54.5,2.84,c,20191216,52.54,0.210959,1.003815,1.037305,0.1,0.000096,0.000184,2.062710e-05,0.000038,-0.000113,-0.000001
4,20191001,55.0,2.62,c,20191216,52.54,0.210959,1.003815,1.046821,0.1,0.000087,0.000165,2.323519e-05,0.000038,-0.000113,-0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,20191001,50.5,2.29,p,20191216,52.54,0.210959,1.003815,0.961172,0.1,0.000090,0.000187,-2.175857e-05,0.000038,-0.000113,-0.000001
154,20191001,51.0,2.47,p,20191216,52.54,0.210959,1.003815,0.970689,0.1,0.000095,0.000196,-1.720263e-05,0.000038,-0.000113,-0.000001
155,20191001,51.5,2.67,p,20191216,52.54,0.210959,1.003815,0.980206,0.1,0.000101,0.000205,-1.219679e-05,0.000038,-0.000113,-0.000001
156,20191001,52.0,2.88,p,20191216,52.54,0.210959,1.003815,0.989722,0.1,0.000107,0.000215,-6.636205e-06,0.000038,-0.000113,-0.000001


In [95]:
ss = 'NGM19'
dfs = create_skew_df(ss,20190321,20190422,deltak=.1)
iplot(cvt.plotly_plot(dfs,'date',plot_title=f'{ss} skew'))

2020-01-03 00:08:11,512 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): fred.stlouisfed.org:443
2020-01-03 00:08:11,980 - urllib3.connectionpool - DEBUG - https://fred.stlouisfed.org:443 "GET /graph/fredgraph.csv?id=USD1MTD156N HTTP/1.1" 200 34481


### create dash app

#### define cell styles

In [96]:
CONTRACTS_TO_DISPLAY_DICT = {'names':['E-Mini SP','Nymex Crude','Ice Brent','NYMEX Natural Gas'], 
                             'symbols':['ES','CL','CB','NG']
}                             

# Create css styles for some parts of the display
STYLE_TITLE={
    'line-height': '20px',
    'borderWidth': '1px',
    'borderStyle': 'dashed',
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#21618C',
    'color':'#FFFFF9',
    'vertical-align':'middle',
} 
STYLE_UPGRID = STYLE_TITLE.copy()
STYLE_UPGRID['background-color'] = '#EAEDED'
STYLE_UPGRID['line-height'] = '10px'
STYLE_UPGRID['color'] = '#21618C'
STYLE_UPGRID['height'] = '50px'

ALL_SYMBOL_SQL = 'select distinct symbol from sec_schema.options_table;'
ALL_SYMBOLS = pga.get_sql(ALL_SYMBOL_SQL).symbol.values
ALL_PRODUCTS = sorted(list(set([s[:2] for s in ALL_SYMBOLS])))


# DICT_ALL_SYMBOLS = {s[:2]:{}}

In [97]:
def get_all_years_per_product(product):
    return sorted(list(set([s[3:] for s in ALL_SYMBOLS if s[:2] == product])))
def get_all_monthcodes_per_product(product,year):
    yy = str(year)[-2:]
    return sorted(list(set([s[2] for s in ALL_SYMBOLS if (s[:2] == product) & (s[-2:]==yy)])))
def get_dates_per_symbol(symbol):
    sql_dates = f'''
    select min(settle_date) min_date, max(settle_date) max_date 
    from sec_schema.options_table 
    where symbol='{symbol}'; 
    '''
    df_dates_per_symbol = pga.get_sql(sql_dates)
    min_date_yyyymmdd = int(df_dates_per_symbol.iloc[0].min_date)
    max_date_yyyymmdd = int(df_dates_per_symbol.iloc[0].max_date)
    # subtract 11 days from max_date_yyyymmdd
    max_date_yyyymmdd = yyyymmdd_from_dt(dt_from_yyyymmdd(max_date_yyyymmdd) - datetime.timedelta(11))
    return [min_date_yyyymmdd,max_date_yyyymmdd]

In [98]:
get_dates_per_symbol('CLM19')

[20160906, 20190505]

#### define cells

In [99]:
list(range(2011,2021))

[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [100]:
DEFAULT_PROD = 'CL'
DEFAULT_YEAR = 2020
# Step 1: create a title at the top
title_div = html.Div([html.H3('Commodity Options SKEW INDEX Analysis')],style=STYLE_TITLE)

# Step 2: create a grid that includes an explanation, and 2 dropdowns
#    Step 2.1: create the information h3
m = html.H3("From the dropdown buttons to the right, select a Commodity and/or a Year",style={'height':'1px'})
info_div = dgrid.GridItem(m,html_id='explain')
select_commod_div  = dgrid.DropDownDiv('commod_dropdown', 
                        CONTRACTS_TO_DISPLAY_DICT['names'], 
                         CONTRACTS_TO_DISPLAY_DICT['symbols'],style=STYLE_UPGRID,
                        transformer_method=lambda data: data#_transform_commod_selection
                         )
init_whole_years = [str(i) for i in list(range(2011,2021))]
init_yy = [str(i)[-2:] for i in init_whole_years]
select_year_div =  dgrid.DropDownDiv('year_dropdown',                         
                        init_whole_years,init_yy,
                        style=STYLE_UPGRID,default_initial_index=len(init_whole_years)-1)



init_months = [s for s in 'FGHJKMNQUVXZ']
select_month_div =  dgrid.DropDownDiv('month_dropdown',                         
                        init_months,init_months,
                        style=STYLE_UPGRID,default_initial_index=len(init_months)-1)

dropdown_grid = dgrid.create_grid([info_div,select_commod_div,select_year_div,select_month_div],num_columns=4,column_width_percents=[55,14.95,14.95,14.95])


#### define main grid


In [101]:
def get_symbol_to_show():
    prod = select_commod_div.current_value
    month = select_month_div.current_value
    yy = select_year_div.current_value
    sym = prod+month+yy
    print(f'get_symbol_to_show: {sym}')
    # get all dates for this sym
    beg_end_yyyymmdds = get_dates_per_symbol(sym)
    # get last date
    end_yyyymmdd = beg_end_yyyymmdds[1]
    # make first date 60 days back from last date
    dt_beg = dt_from_yyyymmdd(end_yyyymmdd) - datetime.timedelta(60)
    beg_yyyymmdd = yyyymmdd_from_dt(dt_beg)
    deltak = DICT_DELTAK[prod]
    dfs = create_skew_df(sym,beg_yyyymmdd,end_yyyymmdd,deltak=deltak)
    fig = cvt.plotly_plot(dfs,'date',plot_title=f'{sym} skew')
    gr = dgrid.GridGraph(fig.layout.title, fig.layout.title ,None,figure=fig,
            df_x_column='date')
    # combine the table and the graph into the main grid
    main_grid =  dgrid.create_grid([gr],num_columns=1)
    return main_grid

content_div = dgrid.ReactiveDiv('page_content',select_commod_div.output_tuple,
#                     input_transformer=lambda commod,data:main_grid,
                    input_transformer=lambda commod,data:get_symbol_to_show(),
                    dom_storage_dict=None)


#### define main app

In [235]:
app = dash.Dash(url_base_pathname='/skew/')
main_div = html.Div(children=[title_div,dropdown_grid,content_div.html])

app.layout = html.Div(children=[main_div])

callback_components = [select_commod_div,select_year_div,select_month_div,content_div]
[c.callback(app) for c in callback_components]


# Step 5: run the server    
host = '127.0.0.1'
port = 8600
app.run_server(host=host,port=port)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-02 13:48:47,844 - werkzeug - INFO -  * Running on http://127.0.0.1:8600/ (Press CTRL+C to quit)
2020-01-02 13:48:53,578 - werkzeug - INFO - 127.0.0.1 - - [02/Jan/2020 13:48:53] "GET /skew/ HTTP/1.1" 200 -
2020-01-02 13:48:53,649 - werkzeug - INFO - 127.0.0.1 - - [02/Jan/2020 13:48:53] "GET /skew/_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-02 13:48:53,662 - werkzeug - INFO - 127.0.0.1 - - [02/Jan/2020 13:48:53] "GET /skew/_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-02 13:48:53,666 - werkzeug - INFO - 127.0.0.1 - - [02/Jan/2020 13:48:53] "GET /skew/_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-02 13:48:53,675 - werkzeug - INFO - 127.0.0.1 - - [02/Jan/2020 13:48:53] "GET /skew/_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1574289294 HTTP/1.1" 200 -
2020-01-02 13:48:53,681 - wer

entering commod_dropdown with data ES
entering month_dropdown with data Z
entering year_dropdown with data 20
entering ReactiveDiv callback
get_symbol_to_show: ESZ20


2020-01-02 13:49:01,076 - werkzeug - INFO - 127.0.0.1 - - [02/Jan/2020 13:49:01] "POST /skew/_dash-update-component HTTP/1.1" 200 -


entering year_dropdown with data 19


...

2020-01-02 14:07:54,041 - werkzeug - INFO - 127.0.0.1 - - [02/Jan/2020 14:07:54] "POST /skew/_dash-update-component HTTP/1.1" 200 -


entering month_dropdown with data Z


2020-01-02 14:07:56,623 - werkzeug - INFO - 127.0.0.1 - - [02/Jan/2020 14:07:56] "POST /skew/_dash-update-component HTTP/1.1" 200 -


entering commod_dropdown with data ES
entering ReactiveDiv callback
get_symbol_to_show: ESZ19


2020-01-02 14:08:00,055 - werkzeug - INFO - 127.0.0.1 - - [02/Jan/2020 14:08:00] "POST /skew/_dash-update-component HTTP/1.1" 200 -


### Create Dash App to server up SKEW

In [102]:
sys.path.append(os.path.abspath('../../risktables'))
sys.path.append(os.path.abspath('../../risktables/risktables'))


In [103]:
from risktables import dgrid_components as dgc

In [116]:
import importlib
importlib.reload(dgc)
# dgc.ChainedDropDownDiv??

<module 'risktables.dgrid_components' from '/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/dgrid_components.py'>

In [117]:
get_all_monthcodes_per_product('ES',2019)

['H', 'M', 'U', 'Z']

In [127]:
chained_dd_prods = dgc.ChainedDropDownDiv('chained_dd_prods',
                initial_dropdown_labels=['Emini','WTI Crude','Brent Crude'],
                initial_dropdown_values=['ES','CL','CB'])

def _chained_years(prod):
    if prod is None or len(prod)<1:
        return []
    yys = get_all_years_per_product(prod)
    choices = [{'label':str(2000 + int(yy)),'value':yy} for yy in yys]
    print(f'_chained_years choices: {choices}')
    return  choices

    
chained_dd_years = dgc.ChainedDropDownDiv('chained_dd_years',
                dropdown_input_component=chained_dd_prods,
                choices_transformer_method=_chained_years)

def _chained_months(prod):
#     prod = chained_dd_prods.current_value
    if prod is None or len(prod)<1:
        return []
    yy = chained_dd_years.current_value
    if yy is None or len(yy)<1:
        return []
    year = 2000 + int(yy)
    mcs = get_all_monthcodes_per_product(prod,year)
    choices = [{'label':mc ,'value':mc} for mc in mcs]
    print(f'_chained_months choices: {choices}')
    return  choices

    
chained_dd_months = dgc.ChainedDropDownDiv('chained_dd_months',
                dropdown_input_component=chained_dd_years,
                choices_transformer_method=_chained_months)

In [ ]:
app_to_use = dash.Dash(url_base_pathname='/test/')
# app.layout = html.Div(children=[chained_dd.html])

app_component_list = [chained_dd_prods,chained_dd_years,chained_dd_months]
# gtcl = ['50%','50%']
gtcl = ['1fr 1fr 1fr']
app = dgc.make_app(app_component_list,
                app=app_to_use,
                grid_template_columns_list=gtcl)    


# Step 5: run the server    
host = '127.0.0.1'
port = 8600
app.run_server(host=host,port=port)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-03 00:40:12,073 - werkzeug - INFO -  * Running on http://127.0.0.1:8600/ (Press CTRL+C to quit)
2020-01-03 00:40:15,912 - werkzeug - INFO - 127.0.0.1 - - [03/Jan/2020 00:40:15] "GET /test/ HTTP/1.1" 200 -
2020-01-03 00:40:16,012 - werkzeug - INFO - 127.0.0.1 - - [03/Jan/2020 00:40:16] "GET /test/_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-03 00:40:16,037 - werkzeug - INFO - 127.0.0.1 - - [03/Jan/2020 00:40:16] "GET /test/_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-03 00:40:16,040 - werkzeug - INFO - 127.0.0.1 - - [03/Jan/2020 00:40:16] "GET /test/_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-03 00:40:16,047 - werkzeug - INFO - 127.0.0.1 - - [03/Jan/2020 00:40:16] "GET /test/_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1574289294 HTTP/1.1" 200 -
2020-01-03 00:40:16,085 - wer

_dropdown_transformer chained_dd_months input: [None, None]
_dropdown_transformer chained_dd_years input: [None, None]
_dropdown_transformer chained_dd_prods input: ['ES']
_dropdown_transformer chained_dd_months input: [None, None]
_dropdown_transformer chained_dd_years input: [None, 'ES']
_chained_years choices: [{'label': '2011', 'value': '11'}, {'label': '2012', 'value': '12'}, {'label': '2013', 'value': '13'}, {'label': '2014', 'value': '14'}, {'label': '2015', 'value': '15'}, {'label': '2016', 'value': '16'}, {'label': '2017', 'value': '17'}, {'label': '2018', 'value': '18'}, {'label': '2019', 'value': '19'}, {'label': '2020', 'value': '20'}]


2020-01-03 00:40:17,121 - root - DEBUG - chained_dd_months_dropdown_html input: [None, 'ES']
2020-01-03 00:40:17,123 - root - DEBUG - chained_dd_months_dropdown_html output: ['ES', []]
2020-01-03 00:40:17,124 - werkzeug - INFO - 127.0.0.1 - - [03/Jan/2020 00:40:17] "POST /test/_dash-update-component HTTP/1.1" 200 -


_dropdown_transformer chained_dd_months input: [None, 'ES']


2020-01-03 00:40:21,971 - root - DEBUG - chained_dd_years_dropdown_html input: ['13', 'ES']
2020-01-03 00:40:21,973 - root - DEBUG - chained_dd_years_dropdown_html output: ['ES', [{'label': '2011', 'value': '11'}, {'label': '2012', 'value': '12'}, {'label': '2013', 'value': '13'}, {'label': '2014', 'value': '14'}, {'label': '2015', 'value': '15'}, {'label': '2016', 'value': '16'}, {'label': '2017', 'value': '17'}, {'label': '2018', 'value': '18'}, {'label': '2019', 'value': '19'}, {'label': '2020', 'value': '20'}]]
2020-01-03 00:40:21,975 - werkzeug - INFO - 127.0.0.1 - - [03/Jan/2020 00:40:21] "POST /test/_dash-update-component HTTP/1.1" 200 -
2020-01-03 00:40:22,028 - root - DEBUG - chained_dd_months_dropdown_html input: [None, 'ES']
2020-01-03 00:40:22,029 - root - DEBUG - chained_dd_months_dropdown_html output: ['ES', []]
2020-01-03 00:40:22,031 - werkzeug - INFO - 127.0.0.1 - - [03/Jan/2020 00:40:22] "POST /test/_dash-update-component HTTP/1.1" 200 -


_dropdown_transformer chained_dd_years input: ['13', 'ES']
_chained_years choices: [{'label': '2011', 'value': '11'}, {'label': '2012', 'value': '12'}, {'label': '2013', 'value': '13'}, {'label': '2014', 'value': '14'}, {'label': '2015', 'value': '15'}, {'label': '2016', 'value': '16'}, {'label': '2017', 'value': '17'}, {'label': '2018', 'value': '18'}, {'label': '2019', 'value': '19'}, {'label': '2020', 'value': '20'}]
_dropdown_transformer chained_dd_months input: [None, 'ES']


2020-01-03 00:40:28,289 - root - DEBUG - chained_dd_prods_dropdown_html input: ['CL']
2020-01-03 00:40:28,290 - root - DEBUG - chained_dd_prods_dropdown_html output: ['CL', [{'label': 'Emini', 'value': 'ES'}, {'label': 'WTI Crude', 'value': 'CL'}, {'label': 'Brent Crude', 'value': 'CB'}]]
2020-01-03 00:40:28,292 - werkzeug - INFO - 127.0.0.1 - - [03/Jan/2020 00:40:28] "POST /test/_dash-update-component HTTP/1.1" 200 -
2020-01-03 00:40:28,344 - root - DEBUG - chained_dd_years_dropdown_html input: ['13', 'CL']
2020-01-03 00:40:28,346 - root - DEBUG - chained_dd_years_dropdown_html output: ['CL', [{'label': '2011', 'value': '11'}, {'label': '2012', 'value': '12'}, {'label': '2013', 'value': '13'}, {'label': '2014', 'value': '14'}, {'label': '2015', 'value': '15'}, {'label': '2016', 'value': '16'}, {'label': '2017', 'value': '17'}, {'label': '2018', 'value': '18'}, {'label': '2019', 'value': '19'}, {'label': '2020', 'value': '20'}, {'label': '2021', 'value': '21'}, {'label': '2022', 'value

_dropdown_transformer chained_dd_prods input: ['CL']
_dropdown_transformer chained_dd_years input: ['13', 'CL']
_chained_years choices: [{'label': '2011', 'value': '11'}, {'label': '2012', 'value': '12'}, {'label': '2013', 'value': '13'}, {'label': '2014', 'value': '14'}, {'label': '2015', 'value': '15'}, {'label': '2016', 'value': '16'}, {'label': '2017', 'value': '17'}, {'label': '2018', 'value': '18'}, {'label': '2019', 'value': '19'}, {'label': '2020', 'value': '20'}, {'label': '2021', 'value': '21'}, {'label': '2022', 'value': '22'}, {'label': '2023', 'value': '23'}]
_dropdown_transformer chained_dd_months input: [None, 'CL']


## End